In [64]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Leagues

1 - Brasileirao A \
2 - Premier League \
3 - Serie A TIM \
4 - Bundesliga \
5 - La Liga

In [65]:
ligas = {
    'brasileirao_a': ["Série A", "https://fbref.com/en/comps/24/Serie-A-Stats", 'Brasileirao', 'https://fbref.com/en/comps/24/schedule/Serie-A-Scores-and-Fixtures'],
    'premier_league': ["Premier League", "https://fbref.com/en/comps/9/Premier-League-Stats", 'Premier_League', 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'],
    'serie_a_tim': ["Serie A", "https://fbref.com/en/comps/11/Serie-A-Stats", 'Serie_A_TIM', "https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures"],
    'bundesliga': ["Bundesliga", "https://fbref.com/en/comps/20/Bundesliga-Stats", 'Bundesliga', "https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures"],
    'la_liga': ["La Liga", "https://fbref.com/en/comps/12/La-Liga-Stats", 'La_Liga', "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"]
}


anos = list(range(2023, 2021, -1))

season_atual = 2023
liga_atual = ligas['premier_league']

# Funcoes

In [66]:
def match_history(league, years = [2023], update = None):
  print("Getting the seasons: ", years)
  liga = league[1]
  match_history = []
  for year in years:
    print("Scraping the season: ", year)
    data = requests.get(liga)
    soup = BeautifulSoup(data.text)
    tabela = soup.select('table.stats_table')[0]            #Seleciona a tabela principal
    links = tabela.find_all('a')                            #Procura a Anchor que contem todos os links do time
    links = [link.get('href') for link in links]            #Pega os links (sem o começo deles)
    links = [link for link in links if '/squads/' in link]  #Pega apenas o link 'squads'
    urls = [f"https://fbref.com{link}" for link in links]   #Adiciona o inicio do html

    try:
      prev_season = soup.select('a.prev')[0].get('href')      #Vai para a temporada anterior
      liga = f"https://fbref.com{prev_season}"
    except IndexError:
      continue

    #Itera sobre todos os times da tabela
    for team in urls:
      print(team)
      nome_time = team.split('/')[-1].replace('-Stats', '').replace('-','_').lower()
      data = requests.get(team)
      soup = BeautifulSoup(data.text)

      ##Partidas jogadas
      matches = pd.read_html(data.text, match= 'Scores & Fixtures')[0]

      ##Chutes
      links = [link.get("href") for link in soup.find_all('a')]
      links = [link for link in links if link and 'all_comps/shooting/' in link]
      data_shooting = requests.get(f"https://fbref.com{links[0]}")
      shooting = pd.read_html(data_shooting.text, match= "Shooting")[0]
      shooting.columns = shooting.columns.droplevel()
      time.sleep(2)

      ##Goleiros
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/keeper' in l]
      data_goalkeeping = requests.get(f"https://fbref.com{links[0]}")
      goalkeeping = pd.read_html(data_goalkeeping.text, match= "Goalkeeping")[0]
      goalkeeping.columns = goalkeeping.columns.droplevel()
      time.sleep(2)

      ##Passes
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/passing' in l]
      data_passing = requests.get(f"https://fbref.com{links[0]}")
      passing = pd.read_html(data_passing.text, match= "Passing")[0]
      passing.columns = passing.columns.droplevel()
      time.sleep(2)

      ##Tipos de Passes
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/passing_types' in l]
      data_passtype = requests.get(f"https://fbref.com{links[0]}")
      pass_types = pd.read_html(data_passtype.text, match= "Pass Types")[0]
      pass_types.columns = pass_types.columns.droplevel()
      time.sleep(2)

      ##Gols e Criação de Chutes
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/gca' in l]
      data_shotcreation = requests.get(f"https://fbref.com{links[0]}")
      goal_shotcreation = pd.read_html(data_shotcreation.text, match= "Goal and Shot Creation")[0]
      goal_shotcreation.columns = goal_shotcreation.columns.droplevel()
      time.sleep(2)

      ##Acoes Defensivas
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/defense' in l]
      data_defensive = requests.get(f"https://fbref.com{links[0]}")
      defensive = pd.read_html(data_defensive.text, match= "Defensive Actions")[0]
      defensive.columns = defensive.columns.droplevel()
      time.sleep(2)

      ##Posse de Bola
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/possession' in l]
      data_possession = requests.get(f"https://fbref.com{links[0]}")
      possession = pd.read_html(data_possession.text, match= "Possession")[0]
      possession.columns = possession.columns.droplevel()
      time.sleep(2)

      ##Demais Estatisticas
      links = [l.get("href") for l in soup.find_all('a')]
      links = [l for l in links if l and 'all_comps/misc' in l]
      data_misc = requests.get(f"https://fbref.com{links[0]}")
      misc = pd.read_html(data_misc.text, match= "Miscellaneous Stats")[0]
      misc.columns = misc.columns.droplevel()
      time.sleep(2)

      # Merge das abas
      try:
        matches_df = matches.merge(
          shooting[['Date', 'Sh', 'SoT']], on= 'Date')
        matches_df = matches_df.merge(
          goalkeeping[['Date', 'Saves']], on= 'Date')
        matches_df = matches_df.merge(
          passing[['Date', 'Cmp', 'Att', 'PrgP', 'KP', '1/3']], on= 'Date') 
        matches_df = matches_df.merge(
          pass_types[['Date', 'Sw', 'Crs']], on= 'Date')
        matches_df = matches_df.merge(
          goal_shotcreation[['Date', 'SCA', 'GCA']], on= 'Date')
        matches_df = matches_df.merge(
          defensive[['Date', 'Tkl', 'TklW', 'Def 3rd', 'Att 3rd', 'Blocks', 'Int']], on= 'Date')
        matches_df.rename(columns={'Att 3rd': 'Tkl_Att_3rd',
                                   'Def 3rd': 'Tkl_Def_3rd'}, inplace=True)
        matches_df = matches_df.merge(
          possession[['Date', 'Att 3rd']], on= 'Date')
        matches_df.rename(columns={'Att 3rd': 'Touches_Att_3rd'}, inplace=True)
        matches_df = matches_df.merge(
          misc[['Date', 'Fls', 'Off', 'Recov']], on= 'Date')
      except ValueError:
        continue

      matches_df = matches_df[matches_df["Comp"] == league[0]]
      matches_df['Season'] = year
      matches_df['Team'] = nome_time
      
      match_history.append(matches_df)
      time.sleep(4)
    
  match_history = pd.concat(match_history)
  match_history.columns = [c.lower() for c in match_history.columns]
  if update is not None:
    update.drop(update[update['season'] == year].index, inplace= True)
    update.columns = [c.lower() for c in update.columns]
    match_history = pd.concat([match_history, update], axis = 0)

  return match_history

In [82]:
def standings(league, years= [2023], update = None):
  table = []
  liga = league[1]
  print("Getting the seasons: ", years)
  for year in years:
    print("Scraping the season: ", year)
    data = requests.get(liga)
    soup = BeautifulSoup(data.text)
    table_df = pd.read_html(data.text, match= 'Regular season')[0]
    table_df['Season'] = year
    table_df['league_name'] = league[2]
  
    table.append(table_df)
    time.sleep(10)

    try:
      prev_season = soup.select('a.prev')[0].get('href')      #Vai para a temporada anterior
      liga = f"https://fbref.com{prev_season}"
    except IndexError:
      continue
  
  table = pd.concat(table)
  table.columns = [c.lower() for c in table.columns]
  if update is not None:
    update.drop(update[update['season'] == year].index, inplace= True)
    update.columns = [c.lower() for c in update.columns]
    table = pd.concat([table, update], axis = 0)

  return table

In [86]:
def next_matches(league):
  data = requests.get(league[3])
  matches = pd.read_html(data.text, match= 'Scores & Fixtures')[0]
  matches['league_name'] = league[2]
  matches.columns = [c.lower() for c in matches.columns]
  return matches

# Exports

### Baixar anos anteriores:

In [69]:
historico = []
for liga in ligas:
  print(f'Scraping the league: {ligas[liga][2]}')
  historico.append(match_history(ligas[liga], years= anos))

historico = pd.concat(historico)

historico.to_excel(f'../datasets/xlsx/historico(naotratado).xlsx', index= False)

Scraping the league: Brasileirao
Getting the seasons:  [2023]
Scraping the season:  2023
https://fbref.com/en/squads/d9fdd9d9/Botafogo-RJ-Stats
https://fbref.com/en/squads/abdce579/Palmeiras-Stats
https://fbref.com/en/squads/f98930d1/Bragantino-Stats
https://fbref.com/en/squads/d5ae3703/Gremio-Stats
https://fbref.com/en/squads/639950ae/Flamengo-Stats
https://fbref.com/en/squads/422bb734/Atletico-Mineiro-Stats
https://fbref.com/en/squads/2091c619/Athletico-Paranaense-Stats
https://fbref.com/en/squads/84d9701c/Fluminense-Stats
https://fbref.com/en/squads/5f232eb1/Sao-Paulo-Stats
https://fbref.com/en/squads/a9d0ab0e/Fortaleza-Stats
https://fbref.com/en/squads/6f7e1f03/Internacional-Stats
https://fbref.com/en/squads/f0e6fb14/Cuiaba-Stats
https://fbref.com/en/squads/bf4acd28/Corinthians-Stats
https://fbref.com/en/squads/712c528f/Santos-Stats
https://fbref.com/en/squads/157b7fee/Bahia-Stats
https://fbref.com/en/squads/83f55dbe/Vasco-da-Gama-Stats
https://fbref.com/en/squads/03ff5eeb/Cruzeiro

In [83]:
tabela = []
for liga in ligas:
  print(f'Scraping the league: {ligas[liga][2]}')
  tabela.append(standings(ligas[liga], years= anos))

tabela = pd.concat(tabela)

tabela.to_excel(f'../datasets/xlsx/tabela(naotratado).xlsx', index= False)

Scraping the league: Brasileirao
Getting the seasons:  [2023]
Scraping the season:  2023
Scraping the league: Premier_League
Getting the seasons:  [2023]
Scraping the season:  2023
Scraping the league: Serie_A_TIM
Getting the seasons:  [2023]
Scraping the season:  2023
Scraping the league: Bundesliga
Getting the seasons:  [2023]
Scraping the season:  2023
Scraping the league: La_Liga
Getting the seasons:  [2023]
Scraping the season:  2023


In [89]:
rodadas = []
for liga in ligas:
  print(f'Scraping the league: {ligas[liga][2]}')
  rodadas.append(next_matches(ligas[liga]))

rodadas = pd.concat(rodadas)

rodadas.to_excel(f'../datasets/xlsx/rodadas(naotratado).xlsx', index= False)

Scraping the league: Brasileirao
Scraping the league: Premier_League
Scraping the league: Serie_A_TIM
Scraping the league: Bundesliga
Scraping the league: La_Liga


### Atualizar temporada atual:

In [ ]:
historico_old = pd.read_excel(f'../datasets/xlsx/historico(naotratado).xlsx')

historico = []
for liga in ligas:
  print(f'Scraping the league: {ligas[liga][2]}')
  historico.append(match_history(liga_atual, update= historico_old))

historico = pd.concat(historico)

historico.to_excel(f'datasets/xlsx/historico(naotratado).xlsx', index= False)

In [ ]:
tabela_old = pd.read_excel(f'../datasets/xlsx/tabela(naotratado).xlsx')

tabela = []
for liga in ligas:
  print(f'Scraping the league: {ligas[liga][2]}')
  tabela.append(match_history(liga_atual, update= tabela_old))

tabela = pd.concat(tabela)

tabela.to_excel(f'datasets/xlsx/tabela(naotratado).xlsx', index= False)